In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import schedule
import time
from alpaca_trade_api import REST

# Set API keys for Alpaca Paper Trading (make sure to set these to your own)
API_KEY = 'YOUR-API-KEY'
SECRET_KEY = 'YOUR-SECRET-KEY'

# Create the REST API object
api = REST(API_KEY, SECRET_KEY, base_url='https://paper-api.alpaca.markets')

def calculate_ema(data, period):
    return data['Close'].rolling(window=period).mean()

def calculate_bollinger_bands(data, period):
    std = data['Close'].rolling(window=period).std()
    middle_band = data['Close'].rolling(window=period).mean()
    upper_band = middle_band + (2 * std)
    lower_band = middle_band - (2 * std)
    return upper_band, middle_band, lower_band

def calculate_rsi(data, period):
    delta = data['Close'].diff()
    up = delta.clip(lower=0)
    down = -delta.clip(upper=0)
    avg_gain = up.rolling(window=period).mean()
    avg_loss = down.rolling(window=period).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_macd(data, fastperiod, slowperiod, signalperiod):
    ema_fast = calculate_ema(data, fastperiod)
    ema_slow = calculate_ema(data, slowperiod)
    macd = ema_fast - ema_slow
    signal = calculate_ema(pd.DataFrame(macd), signalperiod)
    histogram = macd - signal
    return macd, signal, histogram

def trade_bot():
    # Define indicator parameters
    ema_period = 20
    rsi_period = 14
    macd_fastperiod = 12
    macd_slowperiod = 26
    macd_signalperiod = 9

    # Download daily stock data for the past 70 days
    data = yf.download("BTC-USD", start="2023-04-01", end="2023-06-09", interval='1d')

    # Calculate indicators
    data['EMA20'] = calculate_ema(data, ema_period)
    data['upper'], data['middle'], data['lower'] = calculate_bollinger_bands(data, ema_period)
    data['RSI'] = calculate_rsi(data, rsi_period)
    data['MACD'], data['MACD_signal'], data['MACD_hist'] = calculate_macd(data, macd_fastperiod, macd_slowperiod, macd_signalperiod)

    # Fill NaNs
    data.fillna(0, inplace=True)

    # Define trading signals
    data['long_signal'] = np.where((data['Close'] > data['EMA20']) &
                                   (data['Close'] < data['upper']) &
                                   (data['RSI'] < 30) &
                                   (data['MACD'] > data['MACD_signal']), 1, 0)
    data['short_signal'] = np.where((data['Close'] < data['EMA20']) &
                                    (data['Close'] > data['lower']) &
                                    (data['RSI'] > 70) &
                                    (data['MACD'] < data['MACD_signal']), 1, 0)

    # Determine positions (1 for long, -1 for short, 0 for hold)
    data['position'] = data['long_signal'] - data['short_signal']

    # Prepare for trading
    if data['position'].iloc[-1] == 1:
        api.submit_order(symbol='BTCUSD', qty=0.15, side='buy', type='limit', time_in_force='gtc', limit_price=data['Close'].iloc[-1])
    elif data['position'].iloc[-1] == -1:
        api.submit_order(symbol='BTCUSD', qty=0.15, side='sell', type='limit', time_in_force='gtc', limit_price=data['Close'].iloc[-1])

    # Print the data for review
    print(data)

schedule.every(15).minutes.do(trade_bot)

while True:
    schedule.run_pending()
    time.sleep(1)
